In [97]:
import sys
from pathlib import Path
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from google.cloud import bigquery

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

In [98]:
%reload_ext autoreload
%autoreload 2

In [99]:
from datetime import datetime, timedelta

# Obtener la fecha de hoy
hoy = datetime.now()

# Calcular el último lunes (inicio de la semana pasada)
ultimo_lunes = (hoy - timedelta(days=hoy.weekday() + 7)).replace(hour=0, minute=0, second=0, microsecond=0)

# Calcular el último domingo (fin de la semana pasada)
ultimo_domingo = ultimo_lunes + timedelta(days=6)

print(f"Última semana completa: {ultimo_lunes.date()} a {ultimo_domingo.date()}")


Última semana completa: 2025-08-04 a 2025-08-10


In [101]:
from src.data_utils import load_raw_data

# Calcular el último lunes y domingo (ya calculados arriba, pero los reutilizamos aquí)
# Descargar y cargar solo los datos de esa semana (flujo integrado)
df = load_raw_data(
    fecha_inicio=ultimo_lunes.strftime("%Y-%m-%d"),
    fecha_fin=ultimo_domingo.strftime("%Y-%m-%d"),
    descargar_bq=True
 )

print("Primeras filas de la semana cargada:")
print(df.head())

Usando parquet más reciente: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250812.parquet
Descargando datos desde BigQuery porque descargar_bq=True o no existe el archivo C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250812.parquet
Iniciando conexión con BigQuery...
Conexión establecida.
Descargando datos de fleca-del-port.fleca_ventas_dia.t_facturas_dia_extendida_2023 ...
Conexión establecida.
Descargando datos de fleca-del-port.fleca_ventas_dia.t_facturas_dia_extendida_2023 ...


c:\Workspace\mlops_fleca_project\.venv\lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Filas descargadas de la segunda tabla: 4259
Guardando archivo en C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250812.parquet ...
Archivo guardado correctamente.
Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250812.parquet
Validando fechas entre 2025-08-04 y 2025-08-10 (7 días)
Total de fechas faltantes: 0
No faltan fechas en el rango especificado.
Primeras filas de la semana cargada:
       fecha     n_factura zona_de_venta              producto     familia  \
0 2025-08-10  No facturado         SALON    7 CEREALS FORMATGE  BOCADILLOS   
2 2025-08-10  No facturado         BARRA        7 CEREALS FUET  BOCADILLOS   
4 2025-08-10  No facturado         SALON  7 CEREALS LLONGANISA  BOCADILLOS   
6 2025-08-10  No facturado         SALON  7 CEREALS LLONGANISA  BOCADILLOS   
7 2025-08-10  No facturado         SALON  7 CEREALS LLONGANISA  BOCADILLOS   

   cantidad  base_imponible  tipo_IVA  total  is_summer_peak  is_easter  
0  